<a href="https://www.kaggle.com/code/vaasini889123/lstmvae-rl?scriptVersionId=239253711" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, RepeatVector, TimeDistributed, Masking, BatchNormalization, Bidirectional
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from gym import spaces

In [ ]:
from tensorflow.keras.layers import Lambda, TimeDistributed
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

In [ ]:
import os
from tensorflow.keras.models import save_model

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, RepeatVector, Layer

In [ ]:
import ast
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from google.colab import files

In [ ]:
import json

In [ ]:
from collections import defaultdict, deque

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
import random
import pickle

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from tqdm import tqdm

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import register_keras_serializable

In [ ]:
data = pd.read_csv('/kaggle/input/lstm-vae/user_day_sequences (3).csv')

In [ ]:
data = data['activity_sequence']

In [ ]:
import ast

def convert_to_int_list(activity_sequence):
    try:
        # Attempt to evaluate the string as a list
        int_list = ast.literal_eval(activity_sequence)

        # Check if all elements are integers and convert
        if all(isinstance(item, int) for item in int_list):
          return int_list
        else:
          return [int(x) for x in int_list]
    except (SyntaxError, ValueError):
        # Handle cases where the string is not a valid list representation
        print(f"Invalid activity sequence: {activity_sequence}")
        return []  # or handle it differently, e.g., raise an exception

# Apply the function to your data
# data['activity_sequence'] = data['activity_sequence'].apply(convert_to_int_list) # Original line causing the error
data = data.apply(convert_to_int_list) # Corrected line to apply function to the Series directly

In [ ]:
# Shuffle DataFrame
df = data.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# Parse strings to lists
sequences = [seq for seq in df]

# Get unique labels
all_labels = set()
for seq in sequences:
    all_labels.update(seq)
labels = sorted(list(all_labels))  # e.g., [0, 1, 2, 3, 5, 7, 8, 10, 11, 12]
V = len(labels)  # Vocabulary size, e.g., 13

In [ ]:
encoder = OneHotEncoder(categories=[labels], handle_unknown='ignore') 
max_len = 86  #longest sequence

# Converting sequences to one-hot encoded arrays
X = []
for seq in sequences:
    seq_array = np.array(seq).reshape(-1, 1)
    one_hot = encoder.fit_transform(seq_array).toarray() 
    # Pad to max_len
    padded = np.pad(one_hot, ((0, max_len - len(seq)), (0, 0)), mode='constant')  # Shape: (max_len, V)
    X.append(padded)
X = np.array(X)

In [ ]:
np.save('X.npy', X)

LSTM VAE FOR FEATURE EXTRACTION

In [ ]:
class SimplifiedLSTMVAE:
    def __init__(self, vocab_size, max_len, hidden_size=32, latent_dim=8):
        self.vocab_size = vocab_size
        self.max_len = max_len
        self.hidden_size = hidden_size
        self.latent_dim = latent_dim
        self.model = self.build_model()

    def sampling(self, args):
        mu, log_var = args
        epsilon = K.random_normal(shape=(K.shape(mu)[0], self.latent_dim))
        return mu + K.exp(0.5 * log_var) * epsilon

    def build_model(self):
        # Encoder
        inputs = Input(shape=(self.max_len, self.vocab_size))
        h = LSTM(self.hidden_size)(inputs)
        mu = Dense(self.latent_dim)(h)
        log_var = Dense(self.latent_dim)(h)
        z = Lambda(self.sampling)([mu, log_var])

        # Decoder
        z_repeated = RepeatVector(self.max_len)(z)
        decoder_out = LSTM(self.hidden_size, return_sequences=True)(z_repeated)
        outputs = TimeDistributed(Dense(self.vocab_size, activation='softmax'))(decoder_out)

        # Model
        vae = Model(inputs, outputs)

        # Add VAE loss via a custom layer
        class VAELossLayer(tf.keras.layers.Layer):
            def __init__(self, max_len, **kwargs):
                super(VAELossLayer, self).__init__(**kwargs)
                self.max_len = max_len

            def call(self, inputs):
                x, x_decoded, mu, log_var = inputs
                # Reconstruction loss
                recon_loss = tf.reduce_mean(
                    tf.keras.losses.categorical_crossentropy(x, x_decoded)
                ) * self.max_len

                # KL divergence
                kl_loss = -0.5 * tf.reduce_mean(1 + log_var - tf.square(mu) - tf.exp(log_var))

                # Add loss to the layer
                self.add_loss(recon_loss + 0.1 * kl_loss)

                # Return the decoded output
                return x_decoded

        # Create and apply the custom loss layer
        loss_layer = VAELossLayer(self.max_len)
        outputs = loss_layer([inputs, outputs, mu, log_var])

        # Update the model
        vae = Model(inputs, outputs)

        return vae

    def get_encoder(self):
        encoder = Model(self.model.input, self.model.layers[3].output)
        return encoder

In [ ]:
X = np.load('/kaggle/working/X.npy')

In [ ]:
V = 12
max_len = 86

In [ ]:
# Train
vae = SimplifiedLSTMVAE(vocab_size=V, max_len=max_len)
vae.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))
vae.model.fit(X, X, epochs=10, batch_size=128, shuffle=False, verbose=1)

In [ ]:
def save_vae_model(vae, save_dir="/kaggle/working/vae_model"):
    """Save all components of the VAE"""
    os.makedirs(save_dir, exist_ok=True)
    
    # Save models
    save_model(vae.model, os.path.join(save_dir, "vae.h5"))
    save_model(vae.get_encoder(), os.path.join(save_dir, "encoder.h5"))

    # Save config
    config = {
        'vocab_size': vae.vocab_size,
        'max_len': vae.max_len,
        'hidden_size': vae.hidden_size,
        'latent_dim': vae.latent_dim
    }
    with open(os.path.join(save_dir, 'config.json'), 'w') as f:
        json.dump(config, f)
    
    print(f"Model saved to {save_dir}")

save_vae_model(vae, "my_vae_model")

In [ ]:
vae.model.fit(X, X, epochs=5 , batch_size=128, shuffle=False, verbose=1)

In [ ]:
def save_vae_model(vae, save_dir="/kaggle/working/vae_model"):
    """Save all components of the VAE"""
    os.makedirs(save_dir, exist_ok=True)
    
    # Save models
    save_model(vae.model, os.path.join(save_dir, "vae.h5"))
    save_model(vae.get_encoder(), os.path.join(save_dir, "encoder.h5"))

    # Save config
    config = {
        'vocab_size': vae.vocab_size,
        'max_len': vae.max_len,
        'hidden_size': vae.hidden_size,
        'latent_dim': vae.latent_dim
    }
    with open(os.path.join(save_dir, 'config.json'), 'w') as f:
        json.dump(config, f)
    
    print(f"Model saved to {save_dir}")

save_vae_model(vae, "my_vae_model_2")

In [ ]:
vae.model.fit(X, X, epochs=5 , batch_size=128, shuffle=False, verbose=1)

In [ ]:
def save_vae_model(vae, save_dir="/kaggle/working/vae_model"):
    """Save all components of the VAE"""
    os.makedirs(save_dir, exist_ok=True)
    
    # Save models
    save_model(vae.model, os.path.join(save_dir, "vae.h5"))
    save_model(vae.get_encoder(), os.path.join(save_dir, "encoder.h5"))

    # Save config
    config = {
        'vocab_size': vae.vocab_size,
        'max_len': vae.max_len,
        'hidden_size': vae.hidden_size,
        'latent_dim': vae.latent_dim
    }
    with open(os.path.join(save_dir, 'config.json'), 'w') as f:
        json.dump(config, f)
    
    print(f"Model saved to {save_dir}")

save_vae_model(vae, "my_vae_model_3")

k fld validation


In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
fold_metrics = []
_, log_var, _ = vae.encoder.predict(X)
for train_idx, val_idx in kf.split(X):
    X_val = X[val_idx]
    # Get reconstructions for validation fold
    recon = vae.model.predict(X_val)
    fold_metrics.append({
        'fold_mse': np.mean(np.square(X_val - recon)),
        'fold_kl': -0.5 * np.mean(1 + log_var - np.square(mu) - np.exp(log_var))
    })

print("Cross-validated metrics:", np.mean(fold_metrics, axis=0))

reconstruction quality detection

In [ ]:
vae, encoder, decoder = load_vae_model("my_saved_vae")

In [ ]:
V=12
max_len=86

In [ ]:
vae = SimplifiedLSTMVAE(vocab_size=V, max_len=max_len)

In [ ]:
max_len=12
vocab_size=86

In [ ]:
class SimplifiedLSTMVAE:
    def __init__(self, vocab_size, max_len, hidden_size=32, latent_dim=8):
        self.vocab_size = vocab_size
        self.max_len = max_len
        self.hidden_size = hidden_size
        self.latent_dim = latent_dim
        self.model = self.build_model()

    def sampling(self, args):
        mu, log_var = args
        epsilon = K.random_normal(shape=(K.shape(mu)[0], self.latent_dim))
        return mu + K.exp(0.5 * log_var) * epsilon

    def build_model(self):
        # Encoder
        inputs = Input(shape=(self.max_len, self.vocab_size))
        h = LSTM(self.hidden_size)(inputs)
        mu = Dense(self.latent_dim)(h)
        log_var = Dense(self.latent_dim)(h)
        z = Lambda(sampling)([mu, log_var])

        # Decoder
        z_repeated = RepeatVector(self.max_len)(z)
        decoder_out = LSTM(self.hidden_size, return_sequences=True)(z_repeated)
        outputs = TimeDistributed(Dense(self.vocab_size, activation='softmax'))(decoder_out)

        # Model
        vae = Model(inputs, outputs)

        # Add VAE loss via a custom layer
        class VAELossLayer(tf.keras.layers.Layer):
            def __init__(self, max_len, **kwargs):
                super(VAELossLayer, self).__init__(**kwargs)
                self.max_len = max_len

            def call(self, inputs):
                x, x_decoded, mu, log_var = inputs
                # Reconstruction loss
                recon_loss = tf.reduce_mean(
                    tf.keras.losses.categorical_crossentropy(x, x_decoded)
                ) * self.max_len

                # KL divergence
                kl_loss = -0.5 * tf.reduce_mean(1 + log_var - tf.square(mu) - tf.exp(log_var))

                # Add loss to the layer
                self.add_loss(recon_loss + 0.1 * kl_loss)

                # Return the decoded output
                return x_decoded

        # Create and apply the custom loss layer
        loss_layer = VAELossLayer(self.max_len)
        outputs = loss_layer([inputs, outputs, mu, log_var])

        # Update the model
        vae = Model(inputs, outputs)

        return vae

    def get_encoder(self):
        encoder = Model(self.model.input, self.model.layers[3].output)
        return encoder

In [ ]:
vae_instance = SimplifiedLSTMVAE(vocab_size=86, max_len=12)
vae_model = vae_instance.model
vae_model.save('my_vae_model_3/vae.h5')

In [ ]:
encoder = load_model(
    'my_vae_model_3/encoder.h5',
    custom_objects={'sampling': sampling}
)

In [ ]:
X = np.load('/kaggle/working/X.npy')

In [ ]:
test_input = tf.random.uniform((1, vae.max_len, vae.vocab_size))
z = encoder.predict(X)

In [ ]:
z.shape

In [ ]:
X.shape

In [ ]:
reconstructed = vae.model.predict(X)
recon_loss = tf.keras.losses.categorical_crossentropy(X, reconstructed)
recon_loss = tf.reduce_mean(recon_loss).numpy()

In [ ]:
recon_loss

rl

In [ ]:
mu_all = encoder.predict(X)

In [ ]:
mu_all.shape

In [ ]:
df_latent = pd.DataFrame(index=range(X.shape[0]))

df_latent['latent_vector'] = list(mu_all)

# Verify latent vectors
print("Latent vector shape:", mu_all.shape)
print("Sample latent vector:", df_latent['latent_vector'].iloc[0])

mu_mean = np.mean(mu_all, axis=0)
mu_std = np.std(mu_all, axis=0)
print("Mu mean:", mu_mean)  
print("Mu std:", mu_std)    

In [ ]:
df_latent.to_pickle('vae_latent_vectors.pkl')  
print("Saved DataFrame to 'vae_latent_vectors.pkl'")
np.save('mu_all.npy', mu_all)
print("Saved latent vectors to 'mu_all.npy'")

REINFORCEMENT LEARNING

In [ ]:
beh = pd.read_csv('/kaggle/input/reqd-data-lstmrl/beh.csv')

In [ ]:
info = pd.read_csv('/kaggle/input/reqd-data-lstmrl/merged_df.csv')
info = info.drop(columns={'Unnamed: 0'})

In [ ]:
roles = pd.read_csv('/kaggle/input/reqd-data-lstmrl/role_mappings_users.csv')

In [ ]:
roles['role'] = (
    roles['role_0'] * 1 + 
    roles['role_1'] * 2 + 
    roles['role_2'] * 4 + 
    roles['role_3'] * 8
)

# Create role lookup dictionary
role_lookup = dict(zip(roles['user'], roles['role']))

In [ ]:
mu_all.shape

In [ ]:
roles.shape

In [ ]:
X_features = pd.read_csv('/kaggle/input/lstm-vae/user_day_sequences (3).csv')

In [ ]:
info

In [ ]:
X_features

In [ ]:
info.iloc[:,:22]

In [ ]:
user_ids = info['user']

In [ ]:
user_ids

In [ ]:
a = info.iloc[:,22:]

In [ ]:
a

In [ ]:
info

In [ ]:
labels=[1,2,3,4,5,6,7,8,9,10,11,12]

In [ ]:
for seq in a:
    print(a)

In [ ]:
type(a)

In [ ]:
def convert_to_int_list(activity_sequence):
    try:
        # Attempt to evaluate the string as a list
        int_list = ast.literal_eval(activity_sequence)

        # Check if all elements are integers and convert
        if all(isinstance(item, int) for item in int_list):
          return int_list
        else:
          return [int(x) for x in int_list]
    except (SyntaxError, ValueError):
        # Handle cases where the string is not a valid list representation
        print(f"Invalid activity sequence: {activity_sequence}")
        return []  # or handle it differently, e.g., raise an exception

# Apply the function to your data
a['aggregated_sequence'] = a['aggregated_sequence'].apply(convert_to_int_list) # Original line causing the error
#data = data.apply(convert_to_int_list) # Corrected line to apply function to the Series directly

In [ ]:
m = a['aggregated_sequence']

In [ ]:
encoder = OneHotEncoder(categories=[labels], handle_unknown='ignore')  # Removed sparse=False
max_len = 86  # Set based on longest sequence or analysis

# Convert sequences to one-hot encoded arrays
X = []
for seq in m:
    seq_array = np.array(seq).reshape(-1, 1)
    one_hot = encoder.fit_transform(seq_array).toarray()  # Convert to dense array using toarray()
    # Pad to max_len
    padded = np.pad(one_hot, ((0, max_len - len(seq)), (0, 0)), mode='constant')  # Shape: (max_len, V)
    X.append(padded)
X = np.array(X)  # Shape: (num_sequences, max_len, V)

In [ ]:
X.shape

In [ ]:
lstm_encoder = load_model(
    'my_vae_model_3/encoder.h5',
    custom_objects={'sampling': sampling}
)

In [ ]:
m.shape

In [ ]:
tensor = tf.convert_to_tensor(X, dtype=tf.float32)

In [ ]:
mu_all = lstm_encoder.predict(tensor)

In [ ]:
np.save('mu_all.npy',mu_all)

In [ ]:
X_combined = np.column_stack([
    info,                # Original features (excluding VAE error)
    np.array([role_lookup[uid] for uid in user_ids]),  # Lookup role for each user-day
    mu_all                 # VAE reconstruction error
])

In [ ]:
X_combined.shape

In [ ]:
np.save('X_combined', X_combined)

In [ ]:
X_combined[:,2:]

In [ ]:
X_combined = X_combined[:,2:]

In [ ]:
X_combined[:,20:21]

In [ ]:
X_combined = np.delete(X_combined, 20, axis=1)

In [ ]:
X_combined[20]

In [ ]:
X_combined.shape

In [ ]:
X_combined[:,:21]

In [ ]:
X_combined[:,20:21]

In [ ]:
scaler = StandardScaler()
X_normalized = X_combined.copy()
X_normalized[:, :21] = scaler.fit_transform(X_combined[:, :21])

In [ ]:
X_normalized

In [ ]:
np.save('X_normalised', X_normalized)

In [ ]:
X_combined[0][]

aligning labels size

In [ ]:
merged_df = pd.read_csv('/kaggle/input/reqd-data-lstmrl/merged_df.csv')

In [ ]:
merged_df

In [ ]:
labels_df = beh.melt(
    id_vars=['user'], 
    var_name='date', 
    value_name='is_anomaly'
)

# Convert to datetime and sort
labels_df['date'] = pd.to_datetime(labels_df['date'])
labels_df = labels_df.sort_values(['user', 'date'])


In [ ]:
labels_df

In [ ]:
merged_df = merged_df.rename(columns = {'date_only':'date'})

In [ ]:
merged_df['date'] = pd.to_datetime(merged_df['date'])

In [ ]:
aligned_df = pd.merge(
    merged_df,
    labels_df,
    on=['user', 'date'],
    how='inner'  # Only keep rows present in both
)

In [ ]:
aligned_df = aligned_df.drop(columns='Unnamed: 0')

In [ ]:
is_anomaly = aligned_df[['is_anomaly']

In [ ]:
is_anomaly

In [ ]:
is_anomaly = is_anomaly.to_numpy()

In [ ]:
np.save('anomaly_labels.npy',is_anomaly)

In [ ]:
X_combined[0][20:]

In [ ]:
X_combined[0]

dqn

In [ ]:
X_combined.shape

In [ ]:
class StateBuilder:
    def __init__(self, window_size=7):
        self.window_size = window_size
        self.user_histories = defaultdict(lambda: deque(maxlen=window_size))
        
    def build_state(self, user_id, current_features):
        """Build state vector from 29-dimensional input features
        
        Args:
            user_id: Unique user identifier
            current_features: Array with:
                - First 20: User-day features
                - 21st: Role value
                - Last 8: Encoded activity sequence
        """
        # Convert to numpy array and validate
        features = np.array(current_features, dtype=np.float32)
        if len(features) != 29:
            raise ValueError(f"Expected 29 features, got {len(features)}")
        
        # Initialize history if empty
        if len(self.user_histories[user_id]) == 0:
            self.user_histories[user_id].extend(
                [np.zeros(20, dtype=np.float32)] * self.window_size
            )
        
        # Update history with first 20 features
        self.user_histories[user_id].append(features[:20])
        
        # Calculate temporal statistics
        history = np.array(self.user_histories[user_id])
        temporal_mean = history.mean(axis=0)
        temporal_std = history.std(axis=0)
        
        # Build final state vector (20 + 1 + 20 + 20 + 8 = 69 dimensions)
        state = np.concatenate([
            features[:20],       # Current features (20)
            [features[20]],      # Role value (1)
            temporal_mean,       # Historical mean (20)
            temporal_std,        # Historical std (20)
            features[21:]        # Activity sequence (8)
        ], dtype=np.float32)
        
        return state

In [ ]:
class DQNAgent:
    def __init__(self, state_dim=69, action_dim=2):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.memory = deque(maxlen=100000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        
        # Main and target networks
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_network()
    
    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu', input_dim=self.state_dim),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(self.action_dim)
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(0.001))
        return model
    
    def update_target_network(self):
        self.target_model.set_weights(self.model.get_weights())
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state, epsilon=None):
        if epsilon is None:
            epsilon = self.epsilon
        if np.random.rand() <= epsilon:
            return random.randrange(self.action_dim)
        q_values = self.model.predict(state[np.newaxis], verbose=0)
        return np.argmax(q_values[0])
    
    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        
        minibatch = random.sample(self.memory, batch_size)
        states = np.array([t[0] for t in minibatch])
        actions = np.array([t[1] for t in minibatch])
        rewards = np.array([t[2] for t in minibatch])
        next_states = np.array([t[3] for t in minibatch])
        dones = np.array([t[4] for t in minibatch])
        
        targets = self.model.predict(states, verbose=0)
        next_q_values = self.target_model.predict(next_states, verbose=0)
        
        for i in range(batch_size):
            targets[i][actions[i]] = rewards[i] if dones[i] else rewards[i] + self.gamma * np.amax(next_q_values[i])
        
        self.model.fit(states, targets, batch_size=batch_size, verbose=0)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [ ]:
class InsiderThreatEnv:
    def __init__(self, X_combined, user_ids, labels):
        self.X = X_combined    # Shape (n_samples, 29)
        self.user_ids = user_ids
        self.labels = labels    # Binary anomaly labels
        self.state_builder = StateBuilder()
        self.current_idx = 0
    
    def reset(self):
        self.current_idx = np.random.randint(0, len(self.X))
        user_id = self.user_ids[self.current_idx]
        return self.state_builder.build_state(user_id, self.X[self.current_idx])
    
    def step(self, action):
        is_anomaly = self.labels[self.current_idx]
        
        # Custom reward function
        if action == 1:  # Predicted anomaly
            reward = 15 if is_anomaly else -7  # Strong penalty for false positives
        else:  # Predicted normal
            reward = -20 if is_anomaly else 2   # Severe penalty for false negatives
        
        # Move to next sample
        self.current_idx = (self.current_idx + 1) % len(self.X)
        next_state = self.state_builder.build_state(
            self.user_ids[self.current_idx],
            self.X[self.current_idx]
        )
        
        return next_state, reward, self.current_idx == 0


In [ ]:
def train_and_evaluate(X_combined, user_ids, labels):
    # Initialize environment and agent
    env = InsiderThreatEnv(X_combined, user_ids, labels)
    agent = DQNAgent()
    
    # Training parameters
    batch_size = 128
    episodes = 1000
    log_interval = 50
    
    # Training loop
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        done = False
        
        while not done:
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            total_reward += reward
            state = next_state
            
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        
        # Logging
        if episode % log_interval == 0:
            print(f"Episode {episode:4d} | "
                  f"Total Reward: {total_reward:7.1f} | "
                  f"Epsilon: {agent.epsilon:.3f} | "
                  f"Memory Size: {len(agent.memory)}")
    
    # Evaluation
    print("\n=== Final Evaluation ===")
    test_episodes = 100
    tp, fp, tn, fn = 0, 0, 0, 0
    
    for _ in range(test_episodes):
        state = env.reset()
        done = False
        while not done:
            action = agent.act(state, epsilon=0)  # Greedy policy
            is_anomaly = env.labels[env.current_idx]
            
            if action == 1 and is_anomaly: tp += 1
            elif action == 1 and not is_anomaly: fp += 1
            elif action == 0 and is_anomaly: fn += 1
            else: tn += 1
            
            _, _, done = env.step(action)
    
    precision = tp / (tp + fp + 1e-10)
    recall = tp / (tp + fn + 1e-10)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-10)
    
    print(f"Precision: {precision:.3f} | Recall: {recall:.3f} | F1: {f1:.3f}")
    print(f"Confusion Matrix:\n[[{tn:4d} {fp:4d}]\n [{fn:4d} {tp:4d}]]")
    
    return agent


In [ ]:
trained_agent = train_and_evaluate(X_combined, user_ids, is_anomaly) 0

In [ ]:
trained_agent.model.save('dqn_model.h5')

dqn

In [ ]:
with open('/kaggle/working/user_ids.pkl', 'rb') as f:
    user_ids = pickle.load(f)

In [ ]:
class DQNAgent:
    def __init__(self, state_dim=39, action_dim=2):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.memory = deque(maxlen=5000)  # Reduced memory size
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_network()
    
    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_dim=self.state_dim),  # Smaller network
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(self.action_dim)
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(0.005))
        return model
    
    def update_target_network(self):
        self.target_model.set_weights(self.model.get_weights())
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state, epsilon=None):
        if epsilon is None:
            epsilon = self.epsilon
        if np.random.rand() <= epsilon:
            return random.randrange(self.action_dim)
        q_values = self.model.predict(state[np.newaxis], verbose=0)
        return np.argmax(q_values[0])
    
    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        
        minibatch = random.sample(self.memory, batch_size)
        states = np.array([t[0] for t in minibatch])
        actions = np.array([t[1] for t in minibatch])
        rewards = np.array([t[2] for t in minibatch])
        next_states = np.array([t[3] for t in minibatch])
        dones = np.array([t[4] for t in minibatch])
        
        targets = self.model.predict(states, verbose=0)
        next_q_values = self.target_model.predict(next_states, verbose=0)
        
        for i in range(batch_size):
            targets[i][actions[i]] = rewards[i] if dones[i] else rewards[i] + self.gamma * np.amax(next_q_values[i])
        
        self.model.fit(states, targets, batch_size=batch_size, verbose=0)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
class InsiderThreatEnv:
    def __init__(self, X_combined, user_ids, labels):
        self.X = X_combined[:, [0,1,2,3,4,5,6,7,8,9,20,21,22,23,24,25,26,27,28]]  # First 10 features + role + 8 activity
        self.user_ids = user_ids
        self.labels = labels
        self.state_builder = StateBuilder()
        self.current_idx = 0
    
    def reset(self):
        self.current_idx = np.random.randint(0, len(self.X))
        user_id = self.user_ids[self.current_idx]
        return self.state_builder.build_state(user_id, self.X[self.current_idx])
    
    def step(self, action):
        is_anomaly = self.labels[self.current_idx]
        
        if action == 1:  # Predicted anomaly
            reward = 15 if is_anomaly else -7
        else:  # Predicted normal
            reward = -20 if is_anomaly else 2
        
        self.current_idx = (self.current_idx + 1) % len(self.X)
        next_state = self.state_builder.build_state(
            self.user_ids[self.current_idx],
            self.X[self.current_idx]
        )
        
        return next_state, reward, self.current_idx == 0

In [ ]:
X_combined[:, [0,1,2,3,4,5,6,7,8,9,20,21,22,23,24,25,26,27,28]][0]

In [ ]:
def train_and_evaluate(X_combined, user_ids, labels):
    env = InsiderThreatEnv(X_combined, user_ids, labels)
    agent = DQNAgent()
    
    batch_size = 64  # Smaller batch size
    episodes = 20   # Fewer episodes
    log_interval = 100
    
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        done = False
        
        while not done:
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            total_reward += reward
            state = next_state
            
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        
        if episode % log_interval == 0:
            print(f"Episode {episode:4d} | Total Reward: {total_reward:7.1f} | Epsilon: {agent.epsilon:.3f}")
    
    # Simplified evaluation
    print("\n=== Final Evaluation ===")
    test_episodes = 50
    tp, fp, tn, fn = 0, 0, 0, 0
    
    for _ in range(test_episodes):
        state = env.reset()
        done = False
        while not done:
            action = agent.act(state, epsilon=0)
            is_anomaly = env.labels[env.current_idx]
            
            if action == 1 and is_anomaly: tp += 1
            elif action == 1 and not is_anomaly: fp += 1
            elif action == 0 and is_anomaly: fn += 1
            else: tn += 1
            
            _, _, done = env.step(action)
    
    precision = tp / (tp + fp + 1e-10)
    recall = tp / (tp + fn + 1e-10)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-10)
    
    print(f"Precision: {precision:.3f} | Recall: {recall:.3f} | F1: {f1:.3f}")
    print(f"Confusion Matrix:\n[[{tn:4d} {fp:4d}]\n [{fn:4d} {tp:4d}]]")
    
    return agent


In [ ]:
X_combined.shape

In [ ]:
X_combined[:,2:]

In [ ]:
X_combined = X_combined[:,2:]

In [ ]:
X_combined = np.delete(X_combined, 20, axis=1)

In [ ]:
X_combined[0]

In [ ]:
X_combined = np.delete(X_combined, [0,1,2,7,9,10,11,13, 18, 19], axis=1)

In [ ]:
X_combined[0]

trained_agent = train_and_evaluate(X_combined, user_ids, is_anomaly)

In [ ]:
import pickle

with open('user_ids.pkl', 'wb') as f:
    pickle.dump(user_ids, f)


In [ ]:
X_combined[0]

ppo from colab improved

In [ ]:
mu_all = np.load('/kaggle/working/mu_all.npy')

In [ ]:
is_anomaly = np.load('/kaggle/working/anomaly_labels.npy')

In [ ]:
X_combined = np.load('/kaggle/working/X_combined_new.npy', allow_pickle=True)

In [ ]:
X_combined[0]

In [ ]:
X_combined = X_combined[:,2]

In [ ]:
env = AnomalyThresholdEnv(
        recon_errors=mu_all,
        role_features=your_role_features,
        labels=is_anomaly
    )
    
    # Train the PPO agent
    ppo_agent = train_ppo_anomaly_detection(env, episodes=500)

In [ ]:
X_combined[0]

In [ ]:
user_ids.shape

In [ ]:
is_anomaly.shape

In [ ]:
np.save('X_combined_new.npy', X_combined)

In [ ]:
def configure_gpu():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Set memory growth before allocating any tensors
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
            return True
        except RuntimeError as e:
            print(f"GPU configuration error: {e}")
    return False

In [ ]:
gpu_available = configure_gpu()

In [ ]:
X_combined[0]

In [ ]:
class AnomalyDetectionEnv:
    def __init__(self, X_combined, user_ids, labels, window_size=7):
        """
        X_combined: (330285, 19) array
            - First 10: manual features
            - Next 1: role value
            - Last 8: LSTM-VAE latent vector
        user_ids: (330285,) user identifiers
        labels: (330285,) anomaly labels (1=anomaly, 0=normal)
        window_size: how many past days to include in state
        """
        self.X = X_combined
        self.user_ids = user_ids
        self.labels = labels
        self.window_size = window_size
        self.current_idx = window_size  # Start with enough history
        
        # Track current user's history
        self.current_user = user_ids[window_size]
        self.user_history = deque(maxlen=window_size)
        for i in range(window_size):
            if user_ids[i] == self.current_user:
                self.user_history.append(X_combined[i])
        
        # Action space: 0 (normal) or 1 (anomaly)
        self.action_space = 2  
        
        # State: current features + historical mean + user role
        self.state_dim = X_combined.shape[1] + X_combined.shape[1]

    def reset(self):
        """Reset to start new episode"""
        self.current_idx = self.window_size
        self.current_user = self.user_ids[self.window_size]
        self.user_history = deque(maxlen=self.window_size)
        for i in range(self.window_size):
            if self.user_ids[i] == self.current_user:
                self.user_history.append(self.X[i])
        return self._get_state()

    def _get_state(self):
        """Create state vector"""
        current = self.X[self.current_idx]
        history_mean = np.mean(self.user_history, axis=0) if self.user_history else np.zeros_like(current)
        role = current[10]  # Role feature is at index 10
        return np.concatenate([current, history_mean, [role]]).astype(np.float32)

    def step(self, action):
        """
        action: 0 (normal) or 1 (anomaly)
        Returns: next_state, reward, done
        """
        true_label = self.labels[self.current_idx]
        
        # Custom reward function
        if action == 1:  # Predicted anomaly
            reward = 1.0 if true_label == 1 else -0.5
        else:  # Predicted normal
            reward = -1.0 if true_label == 1 else 0.1
        
        # Update position and history
        self.current_idx += 1
        if self.current_idx >= len(self.X):
            done = True
            next_state = self.reset()
        else:
            done = False
            # Update user history if same user
            if self.user_ids[self.current_idx] == self.current_user:
                self.user_history.append(self.X[self.current_idx-1])
            else:
                self.current_user = self.user_ids[self.current_idx]
                self.user_history = deque(maxlen=self.window_size)
            next_state = self._get_state()
        
        return next_state, reward, done


In [ ]:
X_combined.shape

In [ ]:


class AnomalyDetectionEnv:
    def __init__(self, X_combined, user_ids, labels, window_size=7):
        """
        X_combined: (330285, 19) array
            - First 10: manual features
            - Next 1: role value
            - Last 8: LSTM-VAE latent vector
        user_ids: (330285,) user identifiers
        labels: (330285,) anomaly labels (1=anomaly, 0=normal)
        window_size: how many past days to include in state
        """
        self.X = X_combined.astype(np.float32)  # Ensure float32 for efficiency
        self.user_ids = user_ids
        self.labels = labels.astype(np.int32)
        self.window_size = window_size
        self.current_idx = window_size
        self.n_features = X_combined.shape[1]
        
        # Track current user's history
        self.current_user = user_ids[window_size]
        self.user_history = deque(maxlen=window_size)
        for i in range(window_size):
            if user_ids[i] == self.current_user:
                self.user_history.append(X_combined[i])
        
        # Action space: 0 (normal) or 1 (anomaly)
        self.action_space = 2
        
        # State: current features + historical mean
        self.state_dim = self.n_features + self.n_features

    def reset(self):
        """Reset to start new episode"""
        self.current_idx = self.window_size
        self.current_user = self.user_ids[self.window_size]
        self.user_history = deque(maxlen=self.window_size)
        for i in range(self.window_size):
            if self.user_ids[i] == self.current_user:
                self.user_history.append(self.X[i])
        return self._get_state()

    def _get_state(self):
        """Create state vector"""
        current = self.X[self.current_idx]
        history_mean = np.mean(list(self.user_history), axis=0) if self.user_history else np.zeros(self.n_features, dtype=np.float32)
        return np.concatenate([current, history_mean]).astype(np.float32)

    def step(self, action):
        """
        action: 0 (normal) or 1 (anomaly)
        Returns: next_state, reward, done
        """
        true_label = self.labels[self.current_idx]
        
        # Optimized reward function
        reward = (
            1.0 if (action == 1 and true_label == 1) else
            -0.5 if (action == 1 and true_label == 0) else
            -1.0 if (action == 0 and true_label == 1) else
            0.1
        )
        
        # Update position
        self.current_idx += 1
        done = self.current_idx >= len(self.X)
        
        if done:
            next_state = self.reset()
        else:
            # Update user history efficiently
            if self.user_ids[self.current_idx] == self.current_user:
                self.user_history.append(self.X[self.current_idx-1])
            else:
                self.current_user = self.user_ids[self.current_idx]
                self.user_history = deque(maxlen=self.window_size)
                self.user_history.append(self.X[self.current_idx-1])
            next_state = self._get_state()
        
        return next_state, reward, done


In [ ]:
class DQNAgent:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.memory = deque(maxlen=10000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 256
        self.model = self._build_model()
        self.target_model = self._build_model()  # Add target network
        self.update_target_model()
        
    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_dim=self.state_dim,
                                kernel_initializer='he_uniform'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(32, activation='relu',
                                kernel_initializer='he_uniform'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(self.action_dim, activation='linear')
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
        return model
    
    def update_target_model(self):
        """Update target network weights"""
        self.target_model.set_weights(self.model.get_weights())
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_dim)
        state = state.reshape(1, -1)
        q_values = self.model.predict(state, verbose=0)[0]
        return np.argmax(q_values)
    
    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        
        minibatch = random.sample(self.memory, self.batch_size)
        
        # Vectorized data preparation
        states = np.array([x[0] for x in minibatch])
        actions = np.array([x[1] for x in minibatch])
        rewards = np.array([x[2] for x in minibatch])
        next_states = np.array([x[3] for x in minibatch])
        dones = np.array([x[4] for x in minibatch])
        
        # Double DQN
        targets = self.model.predict(states, verbose=0)
        next_q_values = self.target_model.predict(next_states, verbose=0)
        
        targets[range(self.batch_size), actions] = rewards + (1 - dones) * self.gamma * np.max(next_q_values, axis=1)
        
        self.model.fit(states, targets, batch_size=self.batch_size, verbose=0)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [ ]:
def train_dqn(X_combined, user_ids, labels, episodes=10):
    env = AnomalyDetectionEnv(X_combined, user_ids, labels)
    agent = DQNAgent(env.state_dim, env.action_space)
    
    rewards_history = []
    target_update_freq = 10  # Update target network every 10 steps
    
    for episode in tqdm(range(episodes), desc="Training DQN"):
        state = env.reset()
        total_reward = 0
        done = False
        step = 0
        
        while not done:
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            
            if len(agent.memory) > agent.batch_size:
                agent.replay()
            
            step += 1
            if step % target_update_freq == 0:
                agent.update_target_model()
        
        rewards_history.append(total_reward)
        
        if episode % 5 == 0:
            print(f"Episode {episode}: Reward={total_reward:.1f}, Epsilon={agent.epsilon:.2f}")
        
        # Memory cleanup
        gc.collect()
    
    return agent, rewards_history

In [ ]:
agent, rewards_history = train_dqn(X_combined, user_ids, is_anomaly, episodes=10)

In [ ]:
class SimpleAnomalyEnv:
    def __init__(self, X_combined, user_ids, labels):
        """
        Simplified environment using:
        - Last 8 columns (latent vectors) as features
        - Column 10 (role) as additional feature
        """
        self.X = X_combined
        self.user_ids = user_ids
        self.labels = labels
        self.current_idx = 0
        
        # Use last 8 columns + role (column 10)
        self.state_dim = 8 + 1  # 8 latent + 1 role
        self.action_space = 2  # 0=normal, 1=anomaly

    def reset(self):
        self.current_idx = random.randint(0, len(self.X)-1)
        return self._get_state()

    def _get_state(self):
        """State: last 8 features + role"""
        latent = self.X[self.current_idx][-8:]  # Last 8 columns
        role = np.array([self.X[self.current_idx][10]])  # Role as array
        return np.concatenate([latent, role]).astype(np.float32)

    def step(self, action):
        true_label = self.labels[self.current_idx]
        
        # Simple reward function
        if action == 1:  # Predict anomaly
            reward = 1.0 if true_label == 1 else -0.5
        else:  # Predict normal
            reward = -1.0 if true_label == 1 else 0.1
        
        self.current_idx = (self.current_idx + 1) % len(self.X)
        done = False  # Continuous training
        
        return self._get_state(), reward, done

class FastDQNAgent:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.memory = deque(maxlen=5000)
        self.gamma = 0.9
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.98
        self.batch_size = 128
        
        # Tiny neural network
        self.model = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_dim=state_dim),
            tf.keras.layers.Dense(action_dim)
        ])
        self.model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(0.001))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_dim)
        q_values = self.model.predict(state[np.newaxis], verbose=0)
        return np.argmax(q_values[0])
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        
        # Vectorized batch processing
        minibatch = random.sample(self.memory, self.batch_size)
        states = np.array([x[0] for x in minibatch])
        actions = np.array([x[1] for x in minibatch])
        rewards = np.array([x[2] for x in minibatch])
        next_states = np.array([x[3] for x in minibatch])
        
        # Single forward pass
        targets = rewards + self.gamma * np.amax(self.model.predict(next_states, verbose=0), axis=1)
        target_vec = self.model.predict(states, verbose=0)
        target_vec[np.arange(self.batch_size), actions] = targets
        
        # Train on batch
        self.model.train_on_batch(states, target_vec)
        
        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [ ]:
def train_fast_dqn(X_combined, user_ids, labels, steps=50000):
    env = SimpleAnomalyEnv(X_combined, user_ids, labels)
    agent = FastDQNAgent(env.state_dim, env.action_space)
    
    state = env.reset()
    reward_history = []
    
    for step in tqdm(range(steps), desc="Training"):
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        reward_history.append(reward)
        
        if len(agent.memory) > agent.batch_size:
            agent.replay()
        
        if step % 1000 == 0:
            avg_reward = np.mean(reward_history[-1000:]) if len(reward_history) > 1000 else 0
            print(f"Step {step}: Avg Reward={avg_reward:.2f}, Epsilon={agent.epsilon:.2f}")
    
    return agent

In [ ]:
agent = train_fast_dqn(X_combined, user_ids, is_anomaly, steps=5000)

In [ ]:
agent.model.save('dqn_model_1.h5')

In [ ]:
# FastDQNAgent class (simplified for evaluation)
class FastDQNAgent:
    def __init__(self, state_dim, action_dim, model_path):
        self.state_dim = state_dim
        self.action_dim = action_dim
        # Specify custom_objects to map 'mse' to the correct loss function
        self.model = tf.keras.models.load_model(
            model_path,
            custom_objects={'mse': tf.keras.losses.MeanSquaredError}
        )
        self.epsilon = 0.0  # No exploration during evaluation

    def act(self, state):
        q_values = self.model.predict(state[np.newaxis], verbose=0)[0]
        return np.argmax(q_values), q_values[1]  # Action and Q-value for action 1

# Evaluation function with sampling
def evaluate_dqn_agent(env, agent, is_anomaly, sample_size=10000):
    # Randomly sample indices
    np.random.seed(42)  # For reproducibility
    sample_indices = np.random.choice(len(is_anomaly), size=sample_size, replace=False)
    sample_indices = np.sort(sample_indices)  # Sort for sequential processing
    
    predictions = []
    true_labels = []
    anomaly_scores = []  # Q-values for action 1 (for AUC-ROC)
    
    for idx in sample_indices:
        env.current_idx = idx
        state = env._get_state()
        action, anomaly_score = agent.act(state)
        predictions.append(action)
        anomaly_scores.append(anomaly_score)
        true_labels.append(is_anomaly[idx])
    
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    auc_roc = roc_auc_score(true_labels, anomaly_scores)
    cm = confusion_matrix(true_labels, predictions)
    tn, fp, fn, tp = cm.ravel()
    
    a = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'false_negatives': fn,
        'true_positives': tp,
        'false_positives': fp,
        'true_negatives': tn,
        'confusion_matrix': cm
    }
    
    return a

In [ ]:
# Initialize environment
env = SimpleAnomalyEnv(X_combined, user_ids, is_anomaly)
    
    # Load saved agent
model_path = '/kaggle/working/dqn_model_1.h5'
agent = FastDQNAgent(env.state_dim, env.action_space, model_path=model_path)
    
# Evaluate on sample
metrics = evaluate_dqn_agent(env, agent, is_anomaly, sample_size=10000)
print("\nEvaluation Metrics (Sampled Data):")
print(metrics)

trying to handle imbalance with class weights

In [ ]:
X_combined = np.load('/kaggle/working/X_combined_new.npy', allow_pickle=True)
is_anomaly = np.load('/kaggle/working/anomaly_labels.npy')


with open('/kaggle/working/user_ids.pkl', 'rb') as f:
    user_ids = pickle.load(f)

In [ ]:
class FullFeatureAnomalyEnv:
    def __init__(self, X_combined, user_ids, labels):
        """
        Environment using all 19 features:
        - Indices 0-9: Statistical (manual) features
        - Index 10: Role
        - Indices 11-18: LSTM autoencoder latent vectors
        """
        if X_combined.shape[1] != 19:
            raise ValueError(f"Expected 19 features, got {X_combined.shape[1]}")
        self.X = X_combined
        self.user_ids = user_ids
        self.labels = labels
        self.current_idx = 0
        self.state_dim = 19  # 10 stat + 1 role + 8 latent
        self.action_space = 2
        print("Initialized environment with state_dim=19, including:")
        print("- First 10 statistical features (indices 0-9)")
        print("- Role feature (index 10)")
        print("- 8 latent vectors (indices 11-18)")

    def reset(self):
        self.current_idx = random.randint(0, len(self.X)-1)
        return self._get_state()

    def _get_state(self):
        return self.X[self.current_idx].astype(np.float32)

    def step(self, action):
        true_label = self.labels[self.current_idx]
        if action == 1:
            reward = 100.0 if true_label == 1 else -10.0
        else:
            reward = -20.0 if true_label == 1 else 1.0
        self.current_idx = (self.current_idx + 1) % len(self.X)
        done = False
        return self._get_state(), reward, done, self.user_ids[self.current_idx]


In [ ]:
class FastDQNAgent:
    def __init__(self, state_dim, action_dim, user_ids, labels, model_path=None):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.memory = deque(maxlen=5000)
        self.user_priorities = deque(maxlen=5000)
        self.gamma = 0.9
        self.epsilon = 1.0 if not model_path else 0.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        self.batch_size = 128
        
        # User-specific anomaly rates
        self.user_anomaly_rates = {}
        unique_users = np.unique(user_ids)
        for user in unique_users:
            user_mask = user_ids == user
            rate = np.mean(labels[user_mask]) if np.sum(user_mask) > 0 else 0.05
            self.user_anomaly_rates[user] = rate
        
        if model_path:
            self.model = tf.keras.models.load_model(
                model_path,
                custom_objects={'mse': tf.keras.losses.MeanSquaredError()}
            )
        else:
            self.model = self._build_model()
        print(f"Model initialized with input shape (None, {state_dim})")

    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_dim=self.state_dim),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(self.action_dim)
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(0.001))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_dim)
        q_values = self.model.predict(state[np.newaxis], verbose=0)[0]
        return np.argmax(q_values)

    def act_batch(self, states):
        q_values = self.model.predict(states, verbose=0)
        return q_values

    def remember(self, state, action, reward, next_state, done, user_id):
        rate = self.user_anomaly_rates[user_id] / 0.05
        priority = max(1e-3, abs(reward) * rate)  # Ensure positive priority
        self.memory.append((state, action, reward, next_state, done))
        self.user_priorities.append(priority)

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        priorities = np.array(self.user_priorities)
        priorities = np.clip(priorities, 1e-3, None)  # Prevent zeros/negatives
        priority_sum = max(priorities.sum(), 1e-6)
        probs = priorities / priority_sum
        try:
            indices = np.random.choice(len(self.memory), self.batch_size, p=probs)
        except ValueError:
            print("Warning: Invalid probabilities, using uniform sampling")
            indices = np.random.choice(len(self.memory), self.batch_size)
        minibatch = [self.memory[i] for i in indices]
        
        states = np.array([x[0] for x in minibatch])
        actions = np.array([x[1] for x in minibatch])
        rewards = np.array([x[2] for x in minibatch])
        next_states = np.array([x[3] for x in minibatch])
        
        targets = rewards + self.gamma * np.amax(self.model.predict(next_states, verbose=0), axis=1)
        target_vec = self.model.predict(states, verbose=0)
        target_vec[np.arange(self.batch_size), actions] = targets
        
        self.model.train_on_batch(states, target_vec)


In [ ]:
def train_fast_dqn(X_combined, user_ids, labels, steps=10000):
    env = FullFeatureAnomalyEnv(X_combined, user_ids, labels)
    agent = FastDQNAgent(env.state_dim, env.action_space, user_ids, labels)
    
    state = env.reset()
    reward_history = []
    
    for step in tqdm(range(steps), desc="Training"):
        action = agent.act(state)
        next_state, reward, done, user_id = env.step(action)
        agent.remember(state, action, reward, next_state, done, user_id)
        state = next_state
        reward_history.append(reward)
        
        if len(agent.memory) > agent.batch_size:
            agent.replay()
        
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)
        
        if step % 1000 == 0:
            avg_reward = np.mean(reward_history[-1000:]) if reward_history else 0
            print(f"Step {step}: Avg Reward={avg_reward:.2f}, Epsilon={agent.epsilon:.3f}")
    
    agent.model.save('/kaggle/working/dqn_model_retrained.h5')
    return agent

In [ ]:
def evaluate_dqn_agent(env, agent, is_anomaly, user_ids, sample_size=10000):
    np.random.seed(42)
    unique_users = np.unique(user_ids)
    indices = []
    samples_per_user = max(1, sample_size // len(unique_users))
    print(f"Sampling ~{samples_per_user} points per user across {len(unique_users)} users")
    for user in unique_users:
        user_indices = np.where(user_ids == user)[0]
        if len(user_indices) > samples_per_user:
            sampled = np.random.choice(user_indices, samples_per_user, replace=False)
        else:
            sampled = user_indices
        indices.extend(sampled)
    indices = np.array(indices)
    if len(indices) > sample_size:
        indices = np.random.choice(indices, sample_size, replace=False)
    elif len(indices) < sample_size:
        remaining = np.random.choice(np.arange(len(is_anomaly)), sample_size - len(indices), replace=False)
        indices = np.concatenate([indices, remaining])
    indices = np.sort(indices)
    
    user_anomaly_rates = {}
    for user in unique_users:
        user_mask = user_ids == user
        rate = np.mean(is_anomaly[user_mask]) if np.sum(user_mask) > 0 else 0.05
        user_anomaly_rates[user] = rate
    print("Computed user-specific anomaly rates (sample):")
    print({k: f"{v:.3f}" for k, v in list(user_anomaly_rates.items())[:5]})
    
    states = env.X[indices].astype(np.float32)
    print(f"Prepared {len(states)} states with shape {states.shape}, using all 19 features")
    true_labels = is_anomaly[indices]
    sample_user_ids = user_ids[indices]
    q_values = agent.act_batch(states)
    anomaly_scores = q_values[:, 1]
    
    predictions = np.zeros_like(anomaly_scores, dtype=np.int32)
    base_threshold = np.median(anomaly_scores)
    for i, user_id in enumerate(sample_user_ids):
        rate = user_anomaly_rates[user_id]
        threshold = base_threshold * (1 - 0.5 * (rate / 0.05))
        predictions[i] = 1 if anomaly_scores[i] > threshold else 0
    
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, zero_division=0)
    recall = recall_score(true_labels, predictions, zero_division=0)
    f1 = f1_score(true_labels, predictions, zero_division=0)
    auc_roc = roc_auc_score(true_labels, anomaly_scores)
    cm = confusion_matrix(true_labels, predictions)
    tn, fp, fn, tp = cm.ravel()
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'false_negatives': fn,
        'true_positives': tp,
        'false_positives': fp,
        'true_negatives': tn,
        'confusion_matrix': cm.tolist()
    }

In [ ]:
print("Training Fast DQN...")
agent = train_fast_dqn(X_combined, user_ids, is_anomaly, steps=5000)

In [ ]:
env = FullFeatureAnomalyEnv(X_combined, user_ids, is_anomaly)
metrics = evaluate_dqn_agent(env, agent, is_anomaly, user_ids, sample_size=10000)
print("\nEvaluation Metrics (User-Aware, Stratified Sample):")
print(metrics)

In [ ]:
env = FullFeatureAnomalyEnv(X_combined, user_ids, is_anomaly)
metrics = evaluate_dqn_agent(env, agent, is_anomaly, user_ids, sample_size=10000)
print("\nEvaluation Metrics (User-Aware, Stratified Sample):")
print(metrics)

In [ ]:
agent.model.save('dqn_agent_2.h5')

above makes model biased towards anomalies, trying to get a balance

In [ ]:
class FullFeatureAnomalyEnv:
    def __init__(self, X_combined, user_ids, labels):
        """
        Environment using all 19 features:
        - Indices 0-9: Statistical (manual) features
        - Index 10: Role
        - Indices 11-18: LSTM autoencoder latent vectors
        """
        if X_combined.shape[1] != 19:
            raise ValueError(f"Expected 19 features, got {X_combined.shape[1]}")
        self.X = X_combined
        self.user_ids = user_ids
        self.labels = labels
        self.current_idx = 0
        self.state_dim = 19
        self.action_space = 2
        print("Initialized environment with state_dim=19, including:")
        print("- First 10 statistical features (indices 0-9)")
        print("- Role feature (index 10)")
        print("- 8 latent vectors (indices 11-18)")

    def reset(self):
        self.current_idx = random.randint(0, len(self.X)-1)
        return self._get_state()

    def _get_state(self):
        return self.X[self.current_idx].astype(np.float32)

    def step(self, action):
        true_label = self.labels[self.current_idx]
        if action == 1:
            reward = 40.0 if true_label == 1 else -10.0
        else:
            reward = -20.0 if true_label == 1 else 30.0
        self.current_idx = (self.current_idx + 1) % len(self.X)
        done = False
        return self._get_state(), reward, done, self.user_ids[self.current_idx]


In [ ]:
# FastDQNAgent class
class FastDQNAgent:
    def __init__(self, state_dim, action_dim, user_ids, labels, model_path=None):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.memory = deque(maxlen=5000)
        self.user_priorities = deque(maxlen=5000)
        self.gamma = 0.9
        self.epsilon = 1.0 if not model_path else 0.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        self.batch_size = 128
        
        self.user_anomaly_rates = {}
        unique_users = np.unique(user_ids)
        for user in unique_users:
            user_mask = user_ids == user
            rate = np.mean(labels[user_mask]) if np.sum(user_mask) > 0 else 0.005
            self.user_anomaly_rates[user] = min(rate, 0.05)  # Cap at 5%
        
        if model_path:
            self.model = tf.keras.models.load_model(
                model_path,
                custom_objects={'mse': tf.keras.losses.MeanSquaredError()}
            )
        else:
            self.model = self._build_model()
        print(f"Model initialized with input shape (None, {state_dim})")

    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_dim=self.state_dim),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(self.action_dim)
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(0.001))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_dim)
        q_values = self.model.predict(state[np.newaxis], verbose=0)[0]
        return np.argmax(q_values)

    def act_batch(self, states):
        q_values = self.model.predict(states, verbose=0)
        return q_values

    def remember(self, state, action, reward, next_state, done, user_id):
        rate = self.user_anomaly_rates[user_id] / 0.005
        priority = max(1e-2, abs(reward) * rate)
        self.memory.append((state, action, reward, next_state, done))
        self.user_priorities.append(priority)

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        priorities = np.array(self.user_priorities)
        priorities = np.clip(priorities, 1e-2, None)
        priority_sum = max(priorities.sum(), 1e-6)
        probs = priorities / priority_sum
        try:
            indices = np.random.choice(len(self.memory), self.batch_size, p=probs)
        except ValueError:
            print("Warning: Invalid probabilities, using uniform sampling")
            indices = np.random.choice(len(self.memory), self.batch_size)
        minibatch = [self.memory[i] for i in indices]
        
        states = np.array([x[0] for x in minibatch])
        actions = np.array([x[1] for x in minibatch])
        rewards = np.array([x[2] for x in minibatch])
        next_states = np.array([x[3] for x in minibatch])
        
        targets = rewards + self.gamma * np.amax(self.model.predict(next_states, verbose=0), axis=1)
        target_vec = self.model.predict(states, verbose=0)
        target_vec[np.arange(self.batch_size), actions] = targets
        
        self.model.train_on_batch(states, target_vec)


In [ ]:
# Training function
def train_fast_dqn(X_combined, user_ids, labels, steps=20000):
    env = FullFeatureAnomalyEnv(X_combined, user_ids, labels)
    agent = FastDQNAgent(env.state_dim, env.action_space, user_ids, labels)
    
    state = env.reset()
    reward_history = []
    
    for step in tqdm(range(steps), desc="Training"):
        action = agent.act(state)
        next_state, reward, done, user_id = env.step(action)
        agent.remember(state, action, reward, next_state, done, user_id)
        state = next_state
        reward_history.append(reward)
        
        if len(agent.memory) > agent.batch_size:
            agent.replay()
        
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)
        
        if step % 1000 == 0:
            avg_reward = np.mean(reward_history[-1000:]) if reward_history else 0
            print(f"Step {step}: Avg Reward={avg_reward:.2f}, Epsilon={agent.epsilon:.3f}")
    
    agent.model.save('/kaggle/working/dqn_model_2.h5')
    return agent


In [ ]:
# Evaluation function with anomaly oversampling
def evaluate_dqn_agent(env, agent, is_anomaly, user_ids, sample_size=10000, target_anomaly_rate=0.05):
    np.random.seed(42)
    # Oversample anomalies
    anomaly_indices = np.where(is_anomaly == 1)[0]
    normal_indices = np.where(is_anomaly == 0)[0]
    n_anomalies = int(sample_size * target_anomaly_rate)  # e.g., 500 for 5%
    n_normal = sample_size - n_anomalies
    
    # Stratified sampling per user
    unique_users = np.unique(user_ids)
    samples_per_user = max(1, (sample_size - n_anomalies) // len(unique_users))
    normal_sampled = []
    for user in unique_users:
        user_normal = np.where((user_ids == user) & (is_anomaly == 0))[0]
        if len(user_normal) > samples_per_user:
            sampled = np.random.choice(user_normal, samples_per_user, replace=False)
        else:
            sampled = user_normal
        normal_sampled.extend(sampled)
    
    # Adjust normal samples
    if len(normal_sampled) > n_normal:
        normal_sampled = np.random.choice(normal_sampled, n_normal, replace=False)
    elif len(normal_sampled) < n_normal:
        extra = np.random.choice(normal_indices, n_normal - len(normal_sampled), replace=False)
        normal_sampled.extend(extra)
    
    # Sample anomalies
    anomaly_sampled = np.random.choice(anomaly_indices, n_anomalies, replace=False) if len(anomaly_indices) >= n_anomalies else anomaly_indices
    
    indices = np.concatenate([normal_sampled, anomaly_sampled])
    np.random.shuffle(indices)
    indices = np.sort(indices)
    
    user_anomaly_rates = {}
    for user in unique_users:
        user_mask = user_ids == user
        rate = np.mean(is_anomaly[user_mask]) if np.sum(user_mask) > 0 else 0.005
        user_anomaly_rates[user] = min(rate, 0.05)
    print("Computed user-specific anomaly rates (sample):")
    print({k: f"{v:.3f}" for k, v in list(user_anomaly_rates.items())[:5]})
    
    states = env.X[indices].astype(np.float32)
    print(f"Prepared {len(states)} states with shape {states.shape}, using all 19 features")
    true_labels = is_anomaly[indices]
    sample_user_ids = user_ids[indices]
    q_values = agent.act_batch(states)
    anomaly_scores = q_values[:, 1]
    
    predictions = np.zeros_like(anomaly_scores, dtype=np.int32)
    base_threshold = np.median(anomaly_scores)
    for i, user_id in enumerate(sample_user_ids):
        rate = user_anomaly_rates[user_id]
        threshold = base_threshold * (1 - 0.1 * (rate / 0.005))
        predictions[i] = 1 if anomaly_scores[i] > threshold else 0
    
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, zero_division=0)
    recall = recall_score(true_labels, predictions, zero_division=0)
    f1 = f1_score(true_labels, predictions, zero_division=0)
    auc_roc = roc_auc_score(true_labels, anomaly_scores)
    cm = confusion_matrix(true_labels, predictions)
    tn, fp, fn, tp = cm.ravel()
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'false_negatives': fn,
        'true_positives': tp,
        'false_positives': fp,
        'true_negatives': tn,
        'confusion_matrix': cm.tolist()
    }


In [ ]:
# Validate data
print(f"Overall anomaly rate: {np.mean(is_anomaly):.4f}")
unique_users = np.unique(user_ids)
outlier_users = [u for u in unique_users if np.mean(is_anomaly[user_ids == u]) > 0.1]
print(f"Users with anomaly rate >10%: {len(outlier_users)}")
    

In [ ]:
# Train
print("Training Fast DQN...")
agent = train_fast_dqn(X_combined, user_ids, is_anomaly, steps=10000)

In [ ]:
# Evaluate
print("\nEvaluating Retrained Agent...")
env = FullFeatureAnomalyEnv(X_combined, user_ids, is_anomaly)
metrics = evaluate_dqn_agent(env, agent, is_anomaly, user_ids, sample_size=10000)
print("\nEvaluation Metrics (User-Aware, Stratified Sample):")
print(metrics)

role based + user based adaptive thresholding

In [ ]:
# Hyperparameters
STATE_DIM = 19 
ROLE_COL_IDX = 10  

In [ ]:
class FullFeatureAnomalyEnv:
    def __init__(self, X_combined, user_ids, labels):
        """
        Environment using all 19 features:
        - Indices 0-9: Statistical (manual) features
        - Index 10: Role
        - Indices 11-18: LSTM autoencoder latent vectors
        """
        if X_combined.shape[1] != 19:
            raise ValueError(f"Expected 19 features, got {X_combined.shape[1]}")
        self.X = X_combined
        self.user_ids = user_ids
        self.labels = labels
        self.current_idx = 0
        self.state_dim = 19
        self.action_space = 2
        print("Initialized environment with state_dim=19, including:")
        print("- First 10 statistical features (indices 0-9)")
        print("- Role feature (index 10)")
        print("- 8 latent vectors (indices 11-18)")

    def reset(self):
        self.current_idx = random.randint(0, len(self.X)-1)
        return self._get_state()

    def _get_state(self):
        return self.X[self.current_idx].astype(np.float32)

    def step(self, action):
        true_label = self.labels[self.current_idx]
        if action == 1:
            reward = 40.0 if true_label == 1 else -10.0
        else:
            reward = -20.0 if true_label == 1 else 30.0
        self.current_idx = (self.current_idx + 1) % len(self.X)
        done = False

        return self._get_state(), reward, done, self.X[self.current_idx][10]  # Return role instead of user_id

In [ ]:
class FastDQNAgent:
    def __init__(self, state_dim, action_dim, roles, labels, model_path=None):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.memory = deque(maxlen=5000)
        self.role_priorities = deque(maxlen=5000)
        self.gamma = 0.9
        self.epsilon = 1.0 if not model_path else 0.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        self.batch_size = 128
        
        # Role-based anomaly rates
        self.role_anomaly_rates = {}
        unique_roles = np.unique(roles)
        for role in unique_roles:
            role_mask = roles == role
            rate = np.mean(labels[role_mask]) if np.sum(role_mask) > 0 else 0.005
            self.role_anomaly_rates[role] = min(rate, 0.05)  # Cap at 5%
        
        if model_path:
            self.model = tf.keras.models.load_model(
                model_path,
                custom_objects={'mse': tf.keras.losses.MeanSquaredError()}
            )
        else:
            self.model = self._build_model()
        print(f"Model initialized with input shape (None, {state_dim})")

    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_dim=self.state_dim),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(self.action_dim)
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(0.001))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_dim)
        q_values = self.model.predict(state[np.newaxis], verbose=0)[0]
        return np.argmax(q_values)

    def act_batch(self, states):
        q_values = self.model.predict(states, verbose=0)
        return q_values

    def remember(self, state, action, reward, next_state, done, role):
        rate = self.role_anomaly_rates[role] / 0.005
        priority = max(1e-2, abs(reward) * rate)
        self.memory.append((state, action, reward, next_state, done))
        self.role_priorities.append(priority)

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        priorities = np.array(self.role_priorities)
        priorities = np.clip(priorities, 1e-2, None)
        priority_sum = max(priorities.sum(), 1e-6)
        probs = priorities / priority_sum
        try:
            indices = np.random.choice(len(self.memory), self.batch_size, p=probs)
        except ValueError:
            print("Warning: Invalid probabilities, using uniform sampling")
            indices = np.random.choice(len(self.memory), self.batch_size)
        minibatch = [self.memory[i] for i in indices]
        
        states = np.array([x[0] for x in minibatch])
        actions = np.array([x[1] for x in minibatch])
        rewards = np.array([x[2] for x in minibatch])
        next_states = np.array([x[3] for x in minibatch])
        
        targets = rewards + self.gamma * np.amax(self.model.predict(next_states, verbose=0), axis=1)
        target_vec = self.model.predict(states, verbose=0)
        target_vec[np.arange(self.batch_size), actions] = targets
        
        self.model.train_on_batch(states, target_vec)

    def get_threshold(self, scores, role):
        """Role-based adaptive threshold"""
        base = np.percentile(scores, 80)  # Fixed percentile
        role_rate = min(self.role_anomaly_rates[role], 0.02)  # Cap at 2%
        adjustment = 0.03 * role_rate  # Simplified adjustment
        return max(0.5 * base, base * (1 - adjustment))

    def predict(self, X):
        """Predict anomalies with role-based thresholding"""
        scores = self.model.predict(X, verbose=0)[:, 1]  # Q-value for anomaly
        predictions = np.zeros(len(X), dtype=int)
        roles = X[:, 10]  # Role feature
        for i, (score, role) in enumerate(zip(scores, roles)):
            threshold = self.get_threshold(scores, role)
            predictions[i] = 1 if score > threshold else 0
        return predictions, scores

In [ ]:
agent = train_fast_dqn(X_combined, user_ids, is_anomaly, steps=8000)

In [ ]:
def evaluate_role_aware(agent, X_test, y_test):
    """
    Evaluates model with role-specific metrics
    Returns: Overall metrics + per-role breakdown
    """
    # Get predictions and scores
    preds, scores = agent.predict(X_test)
    roles = X_test[:, 10]  # Role feature column
    
    # Overall metrics
    metrics = {
        'accuracy': accuracy_score(y_test, preds),
        'precision': precision_score(y_test, preds, zero_division=0),
        'recall': recall_score(y_test, preds, zero_division=0),
        'f1': f1_score(y_test, preds, zero_division=0),
        'auc_roc': roc_auc_score(y_test, scores),
        'confusion_matrix': confusion_matrix(y_test, preds),
    }
    
    # Per-role metrics
    role_metrics = {}
    for role in np.unique(roles):
        mask = (roles == role)
        if sum(mask) > 0:  # Avoid division by zero
            role_metrics[role] = {
                'precision': precision_score(y_test[mask], preds[mask], zero_division=0),
                'recall': recall_score(y_test[mask], preds[mask], zero_division=0),
                'f1': f1_score(y_test[mask], preds[mask], zero_division=0),
                'support': sum(mask),
                'threshold': agent.get_threshold(scores[mask], role)  # Show actual threshold used
            }
    
    return {'overall': metrics, 'role_wise': role_metrics}


In [ ]:
test_metrics = evaluate_role_aware(agent, X_combined, is_anomaly)
print("Overall Metrics:", test_metrics['overall'])

In [ ]:
def analyze_thresholds(agent, X_test):
    """Analyzes threshold distribution across roles"""
    scores = agent.model.predict(X_test, verbose=0)[:, 1]
    roles = X_test[:, 10]
    
    threshold_data = []
    for role in np.unique(roles):
        role_scores = scores[roles == role]
        if len(role_scores) > 0:
            threshold = agent.get_threshold(role_scores, role)
            threshold_data.append({
                'role': role,
                'avg_score': np.mean(role_scores),
                'threshold': threshold,
                'anomaly_rate': agent.role_anomaly_rates.get(role, 0)
            })
    
    return pd.DataFrame(threshold_data)

In [ ]:
X_combined = X_combined.astype(np.float32)  # Convert ALL features to float32
is_anomaly = is_anomaly.astype(np.int32)    # Labels as 0/1 integers

In [ ]:
threshold_df = analyze_thresholds(agent, X_combined)
print(threshold_df.sort_values('threshold'))

baseline models

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

In [ ]:

X_train, X_test, y_train, y_test= train_test_split(
        X_combined, is_anomaly, test_size=0.2, stratify=is_anomaly
    )
    
    # 1. Isolation Forest
iso = IsolationForest(contamination=0.004, random_state=42)
iso_preds = iso.fit_predict(X_train)
iso_preds = [1 if x == -1 else 0 for x in iso_preds]
    
# 2. One-Class SVM
svm = OneClassSVM(nu=0.004)
svm.fit(X_train[y_train == 0])  # Train only on normals
svm_preds = svm.predict(X_train)
svm_preds = [1 if x == -1 else 0 for x in svm_preds]
    

    

In [ ]:
from sklearn.metrics import classification_report
print("\nIsolation Forest:")
print(classification_report(y_train, iso_preds, target_names=["Normal", "Anomaly"]))
    
print("\nOne-Class SVM:")
print(classification_report(y_train, svm_preds, target_names=["Normal", "Anomaly"]))